# Imports and Preprocessing

In [31]:
import os
import sys
import pandas as pd

folder_path = os.path.join(os.path.dirname(os.getcwd()), 'Data_Test_v2')
print(folder_path)
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\Data_Test_v2


# Demand Forecasting with ANN Model

In [32]:
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
from scipy.stats import reciprocal
import numpy as np

# Define function to create the ANN model 
def create_model(n_hidden, n_neurons, learning_rate, activation, input_shape=[3]): 
    model = Sequential()
    model.add(Dense(n_neurons, activation="relu", input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu"))
    model.add(Dense(1))
    optimizer = Adam() #learning_rate=learning_rate_input
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    return model

# Define function to build the model
def model_builder(n_hidden=1, n_neurons=30, learning_rate=3e-3 , activation = 'relu'):
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons, learning_rate=learning_rate, activation=activation) 

# Define the hyperparameters to search for the Randomized Search
param_distribs = {
    "n_hidden": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2),
    "batch_size": [16, 32, 64, 128],
    "epochs": [10, 20, 30, 40, 50],
    "activation": ['relu', 'sigmoid', 'tanh']
}

# Create the model and optimize with Randomized Search
model_ANN = model_builder()
rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distribs, n_iter=10, cv=3, scoring='neg_mean_squared_error')


# Iterate over all items and predict values
target_pred_ANN = pd.DataFrame()
train_pred_ANN = pd.DataFrame()

for column in target_train:

    # Fit the model on the training data
    rnd_search_cv_ANN.fit(data_train, target_train[column])
    print(rnd_search_cv_ANN.best_params_)

    # Predict the target values and add them as new columns to the DataFrames
    target_pred_ANN[column] = rnd_search_cv_ANN.predict(data_test)
    train_pred_ANN[column] = rnd_search_cv_ANN.predict(data_train)


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 32, 'epochs': 40, 'learning_rate': 0.002866036358819917, 'n_hidden': 7, 'n_neurons': 59}


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'tanh', 'batch_size': 32, 'epochs': 30, 'learning_rate': 0.003111536933370332, 'n_hidden': 6, 'n_neurons': 58}


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'tanh', 'batch_size': 64, 'epochs': 40, 'learning_rate': 0.00017708061969273314, 'n_hidden': 8, 'n_neurons': 38}


In [33]:
print(target_pred_ANN)

            sales        sales2        sales3
0   178571.906250  68329.468750  33226.757812
1   178346.984375  68315.734375  33237.648438
2   178285.890625  68280.304688  33212.328125
3   178234.765625  68221.593750  33192.039062
4   178687.765625  68400.898438  33252.878906
5   177858.609375  68059.648438  33177.574219
6   178241.781250  68260.445312  33235.089844
7   178292.296875  68279.757812  33236.851562
8   178432.390625  68327.304688  33210.082031
9   178679.171875  68412.687500  33264.839844
10  178542.562500  68387.546875  33282.140625
11  178287.312500  68268.820312  33229.316406
12  178890.343750  68517.546875  33278.449219
13  178154.453125  68175.820312  33217.101562
14  178304.468750  68245.250000  33205.394531
15  178311.484375  68284.085938  33248.449219
16  178701.171875  68445.187500  33258.476562
17  178687.625000  68446.031250  33262.906250
18  178103.953125  68156.507812  33215.339844
19  178610.828125  68422.679688  33278.515625
20  178769.406250  68480.320312  3

# Demand Forecasting with DT (LightGBM)

In [34]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Create the model and define the hyperparameters to search for the Randomized Search
model_DT = lgb.LGBMRegressor()
param_distribs = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.5],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
}

# Create the Randomized Search object
rnd_search_cv_DT = RandomizedSearchCV(model_DT, param_distribs, n_iter=10, cv=3, scoring='neg_mean_squared_error')

# Iterate over all items and predict values
target_pred_DT = pd.DataFrame()
train_pred_DT = pd.DataFrame()

for column in target_train:

    # Fit the model on the training data
    rnd_search_cv_DT.fit(data_train, target_train[column])
    print(rnd_search_cv_ANN.best_params_)

    # Get the best parameters and best estimator
    best_params = rnd_search_cv_DT.best_params_
    best_estimator = rnd_search_cv_DT.best_estimator_

    # Predict the target values and add them as new columns to the DataFrames
    target_pred_DT[column] = best_estimator.predict(data_test)
    train_pred_DT[column] = best_estimator.predict(data_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000216 seconds.
You can set `force_col_wise=true` to remove the overhead.


[LightGBM] [Info] Total Bins 50
[LightGBM] [Info] Number of data points in the train set: 138, number of used features: 3
[LightGBM] [Info] Start training from score 180447.413043
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

# Solving Newsvendor (Non-parametric)

In [69]:
final_order_quantities_ANN = pd.DataFrame(target_pred_ANN)
final_order_quantities_DT = pd.DataFrame(target_pred_DT)

# Solving Newsvendor (model-based)

In [37]:
# Parameters
products = target_train.columns.values.tolist()  # Product names
costs = np.array([2, 3, 1.5])  # Cost per unit
prices = np.array([5, 7, 4])  # Selling price per unit
salvages = np.array([1, 1.5, 0.5])  # Salvage value per unit
capacity = 260000  # Total capacity constraint

num_simulations = 1000  # Number of simulations for estimating expected profit

In [53]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm

# Calculate the forecast error for ANN on the prediction of the training data
forecast_error_ANN_std = []
for column in target_pred_ANN:
    target_train_flatten = target_train[column].values.flatten()
    train_pred_ANN_flatten = train_pred_ANN[column].values.flatten()
    forecast_error_ANN = target_train_flatten - train_pred_ANN_flatten
    forecast_error_ANN_std.append(np.std(forecast_error_ANN))

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN_model = []

# Loop over each week in data_test
for i in range(len(data_test)):

    # Demand predictions for all products
    demand_predcitions = target_pred_ANN.iloc[i].values

    # Function to calculate expected costs
    def expected_total_cost(order_quantities):
        # Calculate overstock and understock probabilities
        overstock_probs = norm.cdf(order_quantities, demand_predcitions, forecast_error_ANN_std)
        understock_probs = 1 - overstock_probs
        # Calculate expected overstock and understock costs
        expected_overstock_costs = overstock_probs * (order_quantities - demand_predcitions) * (costs - salvages)
        expected_understock_costs = understock_probs * (demand_predcitions - order_quantities) * (prices - costs)
        # Sum up expected costs for all products
        total_expected_cost = np.sum(expected_overstock_costs + expected_understock_costs)
        return total_expected_cost

    # Capacity constraint
    def capacity_constraint(order_quantities):
        return capacity - np.sum(order_quantities)

    # Constraints dictionary for minimize function
    constraints = ({'type': 'ineq', 'fun': capacity_constraint})

    # Initial guess: Mean demands
    #initial_guess = demand_predcitions / np.sum(demand_predcitions) * capacity
    initial_guess = np.full(len(products), capacity / len(products))

    # Ensure initial guess complies with the capacity constraint
    if np.sum(initial_guess) > capacity:
        initial_guess = initial_guess * (capacity / np.sum(initial_guess))

    # Optimization bounds: Order quantities cannot be negative
    bounds = [(0, None) for _ in range(len(products))]

    # Minimize the expected total cost
    #result = minimize(expected_total_cost, initial_guess, constraints=constraints, bounds=bounds, options={'disp': True})
    result = minimize(expected_total_cost, initial_guess, constraints=constraints, bounds=bounds, method='SLSQP', options={'disp': True})

    optimized_quantities = result.x
    minimized_cost = result.fun

    final_order_quantities_ANN_model.append(optimized_quantities)

final_order_quantities_ANN_model = pd.DataFrame(np.row_stack(final_order_quantities_ANN_model))


Optimization terminated successfully    (Exit mode 0)
            Current function value: 35640.25621339922
            Iterations: 31
            Function evaluations: 152
            Gradient evaluations: 31
Optimization terminated successfully    (Exit mode 0)
            Current function value: 35077.34587722396
            Iterations: 27
            Function evaluations: 113
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 34777.528227857736
            Iterations: 27
            Function evaluations: 117
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 34458.34775033546
            Iterations: 32
            Function evaluations: 156
            Gradient evaluations: 32
Optimization terminated successfully    (Exit mode 0)
            Current function value: 36170.57398959826
            Iterations: 26
            Function eva

In [60]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm

# Calculate the forecast error for ANN on the prediction of the training data
forecast_error_DT_std = []
for column in target_pred_DT:
    target_train_flatten = target_train[column].values.flatten()
    train_pred_DT_flatten = train_pred_DT[column].values.flatten()
    forecast_error_DT = target_train_flatten - train_pred_DT_flatten
    forecast_error_DT_std.append(np.std(forecast_error_DT))

# Initialize an empty list to store the final order quantities
final_order_quantities_DT_model = []

# Loop over each week in data_test
for i in range(len(data_test)):

    # Demand predictions for all products
    demand_predcitions = target_pred_DT.iloc[i].values

    # Function to calculate expected costs
    def expected_total_cost(order_quantities):
        # Calculate overstock and understock probabilities
        overstock_probs = norm.cdf(order_quantities, demand_predcitions, forecast_error_ANN_std)
        understock_probs = 1 - overstock_probs
        # Calculate expected overstock and understock costs
        expected_overstock_costs = overstock_probs * (order_quantities - demand_predcitions) * (costs - salvages)
        expected_understock_costs = understock_probs * (demand_predcitions - order_quantities) * (prices - costs)
        # Sum up expected costs for all products
        total_expected_cost = np.sum(expected_overstock_costs + expected_understock_costs)
        return total_expected_cost

    # Capacity constraint
    def capacity_constraint(order_quantities):
        return capacity - np.sum(order_quantities)

    # Constraints dictionary for minimize function
    constraints = ({'type': 'ineq', 'fun': capacity_constraint})

    # Initial guess: Mean demands
    initial_guess = demand_predcitions / np.sum(demand_predcitions) * capacity

    # Ensure initial guess complies with the capacity constraint
    if np.sum(initial_guess) > capacity:
        initial_guess = initial_guess * (capacity / np.sum(initial_guess))

    # Optimization bounds: Order quantities cannot be negative
    bounds = [(0, None) for _ in range(len(products))]

    # Minimize the expected total cost
    #result = minimize(expected_total_cost, initial_guess, constraints=constraints, bounds=bounds, options={'disp': True})
    result = minimize(expected_total_cost, initial_guess, constraints=constraints, bounds=bounds, method='SLSQP', options={'disp': True})

    optimized_quantities = result.x
    minimized_cost = result.fun

    final_order_quantities_DT_model.append(optimized_quantities)
    

final_order_quantities_DT_model = pd.DataFrame(np.row_stack(final_order_quantities_DT_model))

    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 62605.75685377017
            Iterations: 27
            Function evaluations: 114
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 88544.5547066424
            Iterations: 22
            Function evaluations: 95
            Gradient evaluations: 22
Optimization terminated successfully    (Exit mode 0)
            Current function value: 40341.61008141847
            Iterations: 21
            Function evaluations: 97
            Gradient evaluations: 21
Optimization terminated successfully    (Exit mode 0)
            Current function value: 94805.35932617297
            Iterations: 26
            Function evaluations: 111
            Gradient evaluations: 26
Optimization terminated successfully    (Exit mode 0)
            Current function value: 46135.129491417814
            Iterations: 24
            Function evalua

In [63]:
print(final_order_quantities_DT_model)
print(target_test)

                0             1             2
0   169679.185104  67909.331886  2.241148e+04
1   179718.541583  67998.769984  1.228269e+04
2   160539.352887  67983.354613  3.147729e+04
3   182230.532875  67988.014994  9.781452e+03
4   163091.967692  67909.297598  2.899873e+04
5   134448.852332  68440.056104  3.330150e+04
6   154998.564944  68438.819183  3.333701e+04
7   159009.956996  68024.696459  3.296535e+04
8   158559.321904  68245.257340  3.319542e+04
9   192124.620516  67875.379484  0.000000e+00
10  176479.576880  67959.812976  1.556061e+04
11  173797.966328  67995.811096  1.820622e+04
12  158911.135022  68076.577352  3.301229e+04
13  163188.050015  68007.429078  2.880452e+04
14  191050.037341  67987.776305  9.621864e+02
15  158376.556174  68318.453938  3.330499e+04
16  158942.653653  68014.051354  3.304329e+04
17  164381.163402  67941.805435  2.767703e+04
18  158526.618189  68275.514902  3.319787e+04
19  192197.236840  67802.763160  0.000000e+00
20  151324.912576  68333.777248  3

# Evaluation

In [66]:
fail_dt = 0
fail_ann = 0
total_amount = 0
for i in range(len(target_test)):
    for j in range(len(target_test.columns)):
        fail_dt += abs(final_order_quantities_DT_model.iloc[i,j] - target_test.iloc[i,j])
        fail_ann += abs(final_order_quantities_ANN_model.iloc[i,j] - target_test.iloc[i,j])
        total_amount += target_test.iloc[i,j]

print(fail_dt)
print(fail_ann)
print(fail_dt/total_amount)
print(fail_ann/total_amount)

1832875.9108009893
1865738.6832925323
0.12120629595932413
0.12337947904563203


In [55]:
# Parameters
products = target_train.columns.values.tolist()  # Product names
costs = np.array([2, 3, 1.5])  # Cost per unit
prices = np.array([5, 7, 4])  # Selling price per unit
salvages = np.array([1, 1.5, 0.5])  # Salvage value per unit
capacity = 260000  # Total capacity constraint

num_simulations = 1000  # Number of simulations for estimating expected profit

In [70]:
# Loop over each week in target_test
overall_costs_ANN = 0
overall_costs_DT = 0
overall_costs_ANN_model = 0
overall_costs_DT_model = 0

for i in range(len(target_test)):
    for j in range(len(target_test.columns)):
        # Calculate understock and overstock costs
        cost_ANN = 0
        cost_DT = 0
        cost_ANN_model = 0
        cost_DT_model = 0
        
        #Costs for non-parametric
        if final_order_quantities_ANN.iloc[i, j] < target_test.iloc[i, j]:
            cost_ANN = (prices[j] - costs[j]) * (target_test.iloc[i, j] - np.round(final_order_quantities_ANN.iloc[i, j]))
        if final_order_quantities_ANN.iloc[i, j] > target_test.iloc[i, j]:
            cost_ANN = (costs[j] - salvages[j]) * (np.round(final_order_quantities_ANN.iloc[i, j]) - target_test.iloc[i, j])
        if final_order_quantities_DT.iloc[i, j] < target_test.iloc[i, j]:
            cost_DT = (prices[j] - costs[j]) * (target_test.iloc[i, j] - np.round(final_order_quantities_DT.iloc[i, j]))
        if final_order_quantities_DT.iloc[i, j] > target_test.iloc[i, j]:
            cost_DT = (costs[j] - salvages[j]) * (np.round(final_order_quantities_DT.iloc[i, j]) - target_test.iloc[i, j])
        
        # Costs for model-based
        if final_order_quantities_ANN_model.iloc[i, j] < target_test.iloc[i, j]:
            cost_ANN_model = (prices[j] - costs[j]) * (target_test.iloc[i, j] - np.round(final_order_quantities_ANN_model.iloc[i, j]))
        if final_order_quantities_ANN_model.iloc[i, j] > target_test.iloc[i, j]:
            cost_ANN_model = (costs[j] - salvages[j]) * (np.round(final_order_quantities_ANN_model.iloc[i, j]) - target_test.iloc[i, j])
        if final_order_quantities_DT_model.iloc[i, j] < target_test.iloc[i, j]:
            cost_DT_model = (prices[j] - costs[j]) * (target_test.iloc[i, j] - np.round(final_order_quantities_DT_model.iloc[i, j]))
        if final_order_quantities_DT_model.iloc[i, j] > target_test.iloc[i, j]:
            cost_DT_model = (costs[j] - salvages[j]) * (np.round(final_order_quantities_DT_model.iloc[i, j]) - target_test.iloc[i, j])

        # Calculate the total costs for the week
        overall_costs_ANN += cost_ANN
        overall_costs_DT += cost_DT
        overall_costs_ANN_model += cost_ANN_model
        overall_costs_DT_model += cost_DT_model
   

# Print the overall profit
print('Overall costs for ANN (non-parametric): ', int(overall_costs_ANN))
print('Overall costs for DT (non-parametric): ', int(overall_costs_DT))
print('Overall costs for ANN (model-based): ', int(overall_costs_ANN_model))
print('Overall costs for DT (model-based): ', int(overall_costs_DT_model))

Overall costs for ANN (non-parametric):  3460074
Overall costs for DT (non-parametric):  991854
Overall costs for ANN (model-based):  5076194
Overall costs for DT (model-based):  4784484
